In [243]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [244]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[3522045004, 1561014003, 1388995006, 981475300..."
1,Name,object,14,0.00,779583,"[LADYBUGS PARKING LOT SWEEPING, JMS6, Inc., O ..."
2,City,object,30,0.00,32581,"[TAYLORSVILLE, WESTFIELD, BUFFALO, HUNTSVILLE,..."
3,State,object,14,0.00,51,"[AR, ND, UT, MT, MN]"
4,Zip,int64,0,0.00,33611,"[90740, 91605, 14304, 6237, 1742]"
5,Bank,object,1559,0.17,5802,"[PNC BANK, NATIONAL ASSOCIATION, METRO AREA DE..."
6,BankState,object,1566,0.17,56,"[RI, CA, OH, MT, HI]"
7,NAICS,int64,0,0.00,1312,"[445299, 339911, 812320, 238220, 488410]"
8,ApprovalDate,object,0,0.00,9859,"[21-Sep-99, 18-Aug-06, 22-Nov-04, 18-Aug-04, 1..."
9,ApprovalFY,object,0,0.00,70,"[2000, 1993, 1990, 1997, 1996]"


In [245]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [246]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [247]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [248]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [249]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [250]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [251]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [252]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [253]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [254]:
df.shape

(675026, 13)

In [255]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[31-33, 42, 23]"
1,Term,int64,0,0.0,377,"[120, 84, 60, 40]"
2,NoEmp,int64,0,0.0,499,"[20, 28, 1, 7, 15]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[20, 2, 45, 0]"
5,RetainedJob,int64,0,0.0,323,"[4, 0, 2]"
6,UrbanRural,int64,0,0.0,3,[1]
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[100000, 180000, 75000, 25000]"


In [256]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [257]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [258]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [259]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [260]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [261]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [262]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [263]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [264]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [265]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [266]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [267]:
df.shape

(460816, 13)

In [268]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[62, 23, 54]"
1,Term,int64,0,0.0,373,"[36, 72, 60, 84]"
2,NoEmp,int64,0,0.0,432,"[1, 8, 12, 6, 7]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,"[0, 2, 3]"
5,RetainedJob,int64,0,0.0,276,"[1, 0, 2]"
6,UrbanRural,int64,0,0.0,3,"[1, 0]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,12824,"[1000000, 120124, 25000, 750000, 472000]"


In [269]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [270]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [271]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [272]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [273]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [274]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [275]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [276]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [277]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [278]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [279]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [280]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [281]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [282]:
df.shape

(675026, 13)

In [283]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[23, 44-45, 54]"
1,Term,int64,0,0.0,377,"[81, 48, 36, 84, 180]"
2,NoEmp,int64,0,0.0,499,"[1, 2, 6, 5, 3]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,216,"[0, 1, 5]"
5,RetainedJob,int64,0,0.0,323,"[1, 0, 40, 50, 7]"
6,UrbanRural,int64,0,0.0,3,"[0, 2, 1]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[30000, 110000, 24000, 50000, 5000]"


In [284]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv


In [285]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [286]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[3134585000, 6759874001, 3457413008, 175652501..."
1,Name,object,14,0.00,779583,"[Fried Turkey, LLC, SPORTS WORLD, MOM'S DELI A..."
2,City,object,30,0.00,32581,"[HOUSTON, LONG BEACH, CHICO, FRISCO, AUGUSTA]"
3,State,object,14,0.00,51,"[TX, ND, MO, GA, PA]"
4,Zip,int64,0,0.00,33611,"[45631, 11354, 19147, 78735, 2886]"
5,Bank,object,1559,0.17,5802,"[U.S. BANK NATIONAL ASSOCIATION, WELLS FARGO B..."
6,BankState,object,1566,0.17,56,"[TX, MA, CA, VA, IA]"
7,NAICS,int64,0,0.00,1312,"[541211, 811112, 453220, 444130, 621210]"
8,ApprovalDate,object,0,0.00,9859,"[17-Mar-93, 16-Apr-04, 6-Jul-06, 17-Nov-99, 13..."
9,ApprovalFY,object,0,0.00,70,"[2003, 1996, 1996, 2008, 2010]"


In [287]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [288]:
df = df.dropna(subset='Name')

In [289]:
df = df.dropna(subset='City')

In [290]:
df = df.dropna(subset='State')

In [291]:
df = df.dropna(subset='Bank')

In [292]:
df = df.dropna(subset='BankState')

In [293]:
is_valide = {1.0, 2.0}
df['NewExist'] = df['NewExist'].isin(is_valide)

In [294]:
is_valide = {'N','Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide)

In [295]:
is_valide = {'N', 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide)

In [296]:
df.MIS_Status.unique()

array(['P I F', 'CHGOFF', nan], dtype=object)

In [297]:
is_valide = {'P I F', 'CHGOFF'}
df['MIS_Status'] = df['MIS_Status'].isin(is_valide)

In [298]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [299]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [ ]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [300]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [301]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)

In [302]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:])

In [303]:
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [304]:
df.shape

(696572, 19)

In [305]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,0,0.0,28860,"[CEDAR RAPIDS, STAFFORD, Mineloa, OVERLAND PAR..."
1,State,object,0,0.0,51,"[CA, CT, KY, MN, MA]"
2,Bank,object,0,0.0,5221,"[KEYBANK NATIONAL ASSOCIATION, CITIZENS BANK N..."
3,BankState,object,0,0.0,55,"[CA, WA, RI, AL]"
4,NAICS,object,0,0.0,20,"[62, 81, 31-33, 44-45, 72]"
5,ApprovalDate,object,0,0.0,7367,"[12-Jul-02, 25-Apr-05, 12-Jan-05, 1-Apr-03, 31..."
6,ApprovalFY,object,0,0.0,29,"[1995, 2007, 1996]"
7,Term,int64,0,0.0,378,"[57, 300, 120, 60, 84]"
8,NoEmp,int64,0,0.0,502,"[2, 28, 1, 3]"
9,NewExist,bool,0,0.0,2,[True]


In [306]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_Pure.csv
